In [1]:
from pyspark.sql import *

In [2]:
spark = SparkSession.builder.master('local[2]').appName('Local file').getOrCreate()

In [3]:
spark

In [ ]:
# Specify the path to the input file and output file
input_file_path = 'path/to/your/input/file.txt'
output_file_path = 'path/to/your/output/output.txt'

# Read data from the input file line by line and remove blank lines
with open(input_file_path, 'r') as input_file:
    lines = [line.strip() for line in input_file if line.strip()]

# Write non-blank lines to the output file
with open(output_file_path, 'w') as output_file:
    output_file.write('\n'.join(lines))

print("Non-blank lines successfully written to '{}'.".format(output_file_path))


In [21]:
# Read data from the input file line by line
with open(input_file_path, 'r') as input_file:
    for line in input_file:
        if len(line) != str(0):
            print(f"{line.strip()}")  # Print or process each line as required
            
        else:
            print("blank")

only
one

line


In [20]:
# Specify the path to the input file
input_file_path = 'data.txt'

# Read data from the input file line by line
with open(input_file_path, 'r') as input_file:
    for line in input_file:
        if len(line) != str(0):
            print(f"**{line.strip()}**")  # Print or process each line as required
            print("****")
            print("**A.** ")
            
        else:
            print("blank")

**only**
****
**A.** 
**one**
****
**A.** 
****
****
**A.** 
**line**
****
**A.** 


In [5]:
input_file_path = 'data.txt'
# output_file_path = 'path/to/your/output/output.txt'

with open(input_file_path, 'r') as input_file:
    data = input_file.read()
    for i in data:
        print(i)
    # print(len(data))

# with open(output_file_path, 'w') as output_file:
#     output_file.write(data)

# print("Data successfully read from '{}' and written to '{}'.".format(input_file_path, output_file_path))


o
n
l
y


o
n
e
 


l
i
n
e


<div dir="ltr" class="reader-article-content reader-article-content--content-blocks">
          
        
      
    <p id="ember36" class="ember-view reader-content-blocks__paragraph">
        When dealing with big data, joining tables or data frames is one of the most common and crucial operations. In this article, we will delve into how Spark handles joins internally and the techniques employed by the Spark engine to make this process efficient. Understanding these internals will prove invaluable in mastering complex join operations.

<!---->    </p>

  

<!---->
<!---->
<!---->        
      
    <p id="ember37" class="ember-view reader-content-blocks__paragraph">
        In Spark, you can expect to encounter five primary types of joins:

<!---->    </p>

  

<!---->
<!---->
<!---->        
      
    <p id="ember38" class="ember-view reader-content-blocks__paragraph">
        <ul><li>Broadcast Hash Join</li><li>Shuffle Hash Join</li><li>Sort Merge Join</li><li>Cartesian Join</li><li>Broadcast Nested Loop Join</li></ul>

<!---->    </p>

  

<!---->
<!---->
<!---->        
      
    <p id="ember39" class="ember-view reader-content-blocks__paragraph">
        We'll explore each of these join types one by one, for a comprehensive understanding.

<!---->    </p>

  

<!---->
<!---->
<!---->        
      
    <p id="ember40" class="ember-view reader-content-blocks__paragraph">
        Note: Throughout this article, we'll be using Databricks sample tables to explore these joins. If you have a free/trial subscription with Databricks, you can use the same code to understand these joins.

<!---->    </p>

  

<!---->
<!---->
<!---->        
      
    <h3 id="ember41" class="ember-view">
        Broadcast Hash Join

<!---->    </h3>

  

<!---->
<!---->
<!---->        
      
    <p id="ember42" class="ember-view reader-content-blocks__paragraph">
        The Broadcast Hash Join is the speedster of Spark joins. As the name suggests, it occurs when one of the data frames or tables is broadcast to all the executor nodes. To make this happen, one side of the data must be small enough to fit entirely in memory. Initially, this table is stored on the Driver node and then broadcast to all the executor nodes.

<!---->    </p>

  

<!---->
<!---->
<!---->        
      
    <p id="ember43" class="ember-view reader-content-blocks__paragraph">
        By default, Spark considers a table for broadcasting if it's under 10 MB in size. You can modify this configuration using the following command:

<!---->    </p>

  

<!---->
<!---->
<!---->        
        <pre>spark.conf.set("spark.sql.autoBroadcastJoinThreshold", [your threshold in Bytes])</pre>
  

<!---->
<!---->
<!---->        
      
    <p id="ember44" class="ember-view reader-content-blocks__paragraph">
        Spark automatically selects the Broadcast Hash Join when it encounters a smaller data side. To disable this automatic application, use the following configuration:

<!---->    </p>

  

<!---->
<!---->
<!---->        
        <pre>spark.conf.set("spark.sql.autoBroadcastJoinThreshold", -1)</pre>
  

<!---->
<!---->
<!---->        
      
    <p id="ember45" class="ember-view reader-content-blocks__paragraph">
        Note: It is highly recommended not to disable this configuration as it is the most efficient method for joining small and large tables.

<!---->    </p>

  

<!---->
<!---->
<!---->        
      
    <p id="ember46" class="ember-view reader-content-blocks__paragraph">
        Tips to make this join happen :

<!---->    </p>

  

<!---->
<!---->
<!---->        
      
    <p id="ember47" class="ember-view reader-content-blocks__paragraph">
        <ol><li>It should be Equi join with one side being smaller</li><li>Hint as broadcast join for one of the side.</li></ol>

<!---->    </p>

  

<!---->
<!---->
<!---->        
        <pre># Way to mention hint for Broadcast Join

largeDF.join(smallDF.hint("broadcast"),largeDF.joinkey == smallDF.joinkey,"inner")</pre>
  

<!---->
<!---->
<!---->        
      
    <p id="ember48" class="ember-view reader-content-blocks__paragraph">
        Sometimes, Spark may fail to detect the smaller table. In such cases, you can use hints to trigger the Broadcast Hash Join. But broadcasting a large data will lead to OOM issue. The Join side with the hint will be broadcasted irrespective of autoBroadcastJoinThreshold, if a broadcast hint is specified on either side of the join. 

<!---->    </p>

  

<!---->
<!---->
<!---->        
      
    <p id="ember49" class="ember-view reader-content-blocks__paragraph">
        Example to understand the broadcast join:

<!---->    </p>

  

<!---->
<!---->
<!---->        
        <pre>orders = spark.read.table("samples.tpch.orders") # Small Table

lineitem = spark.read.table("samples.tpch.lineitem") # Big Table size : 983MiB 

print("orders",orders.count())
print("lineitem",lineitem.count())

print("orders",orders.rdd.getNumPartitions()) 
print("lineitem",lineitem.rdd.getNumPartitions()) 

orders.hint("broadcast").join(lineitem,orders.o_orderkey == lineitem.l_orderkey,"inner").explain()</pre>
  

<!---->
<!---->
<!---->        
        <pre>== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- BroadcastHashJoin [o_orderkey#1825L], [l_orderkey#1859L], Inner, BuildLeft, false
   :- Exchange SinglePartition, EXECUTOR_BROADCAST, [plan_id=2561] ---&gt; All 3 intial partitions are meregd to single partition
   :  +- Filter isnotnull(o_orderkey#1825L)
   :     +- FileScan parquet samples.tpch.orders
   +- Filter isnotnull(l_orderkey#1859L)  ---&gt; Containing 8 partitions intially, no shuflle happend
      +- FileScan parquet samples.tpch.lineitem</pre>
  

<!---->
<!---->
<!----><!---->
        
    <div class="reader-image-block reader-image-block--resize">
      <figure class="reader-image-block__figure">
          
    <div class="ivm-image-view-model   ">
        
    <div class="ivm-view-attr__img-wrapper display-flex">
<!---->          <img src="https://media.licdn.com/dms/image/D5612AQFzom9LDgaiQA/article-inline_image-shrink_1500_2232/0/1697724490460?e=1707350400&amp;v=beta&amp;t=UAIfMlcckWpaWVme_-Nr_KbgZ-38mTdQA4Qo1M5T_nI" loading="lazy" alt="" id="ember50" class="ivm-view-attr__img--centered  reader-image-block__img evi-image lazy-image ember-view">
    </div>
  
          </div>
  

          <figcaption class="display-block mt2 full-width text-body-small-open t-sans text-align-center t-black--light">
            <!---->Broadcast join stage<!---->
          </figcaption>
      </figure>
    </div>
  

<!---->
<!---->        
      
    <h3 id="ember51" class="ember-view">
        Shuffle Hash Joins

<!---->    </h3>

  

<!---->
<!---->
<!---->        
      
    <p id="ember52" class="ember-view reader-content-blocks__paragraph">
        When both tables are large, getting the broadcast join may lead to out-of-memory issues. In that case, Shuffle Hash Join can be a better option. These are quite expensive compared to broadcast joins as they involve both shuffling of data and hashing.

<!---->    </p>

  

<!---->
<!---->
<!---->        
      
    <p id="ember53" class="ember-view reader-content-blocks__paragraph">
        It happens in two phases:

<!---->    </p>

  

<!---->
<!---->
<!---->        
      
    <p id="ember54" class="ember-view reader-content-blocks__paragraph">
        <ol><li>Repartitioning on both tables will happen based on the joining keys using hash partitioning, providing 200 partitions (as per the default shuffle partitions config; default value is 200).</li><li>A hash table is created for the lookup activity, and the join happens based on that.</li></ol>

<!---->    </p>

  

<!---->
<!---->
<!---->        
      
    <p id="ember55" class="ember-view reader-content-blocks__paragraph">
        Currently, most of the Spark systems have made Sort Merge Join their default choice over Shuffle Hash Join because of its consistently better performance compared to Shuffle Hash Joins. For this example, if you want to test the Shuffle Hash Join, use the following command to turn off the automatic Sort Merge Joins.

<!---->    </p>

  

<!---->
<!---->
<!---->        
        <pre>spark.conf.set("spark.sql.join.preferSortMergeJoin","false")</pre>
  

<!---->
<!---->
<!---->        
      
    <p id="ember56" class="ember-view reader-content-blocks__paragraph">
        If you are aware that the data is distributed uniformly over partitions without any skewness, you can consider this join over Sort Merge Join.

<!---->    </p>

  

<!---->
<!---->
<!---->        
      
    <p id="ember57" class="ember-view reader-content-blocks__paragraph">
        Tips to make this join happen:

<!---->    </p>

  

<!---->
<!---->
<!---->        
      
    <p id="ember58" class="ember-view reader-content-blocks__paragraph">
        <ol><li>It should be an Equi-join.</li><li>Hint it as a Shuffle join for one of the sides.</li></ol>

<!---->    </p>

  

<!---->
<!---->
<!---->        
        <pre># Way to mention hint for Shuffle Join

DF1.join(DF2.hint("shuffle"),DF1.joinkey == DF2.joinkey,"inner")</pre>
  

<!---->
<!---->
<!---->        
      
    <p id="ember59" class="ember-view reader-content-blocks__paragraph">
        Example to understand the Shuffle Hash join:

<!---->    </p>

  

<!---->
<!---->
<!---->        
        <pre>spark.conf.set("spark.sql.join.preferSortMergeJoin","false")

orders = spark.read.table("samples.tpch.orders") # Table Size : 272.4MiB 

lineitem = spark.read.table("samples.tpch.lineitem") # Table size : 983MiB 

print("orders",orders.count())
print("lineitem",lineitem.count())

print("orders",orders.rdd.getNumPartitions()) 
print("lineitem",lineitem.rdd.getNumPartitions()) 

orders.join(lineitem,orders.o_orderkey == lineitem.l_orderkey,"inner").explain()</pre>
  

<!---->
<!---->
<!---->        
        <pre>== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- ShuffledHashJoin [o_orderkey#3177L], [l_orderkey#3211L], Inner, BuildLeft
   :- Exchange hashpartitioning(o_orderkey#3177L, 200), ENSURE_REQUIREMENTS, [plan_id=3624]  --&gt; Raprtitioned based on Join Key
   :  +- Filter isnotnull(o_orderkey#3177L)
   :     +- FileScan parquet samples.tpch.orders
   +- Exchange hashpartitioning(l_orderkey#3211L, 200), ENSURE_REQUIREMENTS, [plan_id=3625] --&gt; Raprtitioned based on Join Key
      +- Filter isnotnull(l_orderkey#3211L)
         +- FileScan parquet samples.tpch.lineitem</pre>
  

<!---->
<!---->
<!----><!---->
        
    <div class="reader-image-block reader-image-block--full-width">
      <figure class="reader-image-block__figure">
          
    <div class="ivm-image-view-model   ">
        
    <div class="ivm-view-attr__img-wrapper display-flex">
<!---->          <img src="https://media.licdn.com/dms/image/D5612AQGK7_pbZSa2nw/article-inline_image-shrink_1500_2232/0/1697724735801?e=1707350400&amp;v=beta&amp;t=oUqv3AqoxG0VAx91VT88N2_t8AY9C0w3QMlB3LgR0KA" loading="lazy" alt="" id="ember60" class="ivm-view-attr__img--centered  reader-image-block__img evi-image lazy-image ember-view">
    </div>
  
          </div>
  

          <figcaption class="display-block mt2 full-width text-body-small-open t-sans text-align-center t-black--light">
            <!---->Shuffle Hash Join stage Implementation<!---->
          </figcaption>
      </figure>
    </div>
  

<!---->
<!---->        
      
    <h3 id="ember61" class="ember-view">
        Shuffle Sort Merge Join

<!---->    </h3>

  

<!---->
<!---->
<!---->        
      
    <p id="ember62" class="ember-view reader-content-blocks__paragraph">
        This is more preferable over Shuffle Hash Join, considering its consistency over different cases. These joins happen in three stages:

<!---->    </p>

  

<!---->
<!---->
<!---->        
      
    <p id="ember63" class="ember-view reader-content-blocks__paragraph">
        <ol><li>Repartitioning on both tables will happen based on the joining keys using hash partitioning, providing 200 partitions (as per the default shuffle partitions config; default value is 200).</li><li>Sorting of data happens on each partition individually.</li><li>As a last step, merging happens based on the joining key.</li></ol>

<!---->    </p>

  

<!---->
<!---->
<!---->        
      
    <p id="ember64" class="ember-view reader-content-blocks__paragraph">
        Tips to make this join happen:

<!---->    </p>

  

<!---->
<!---->
<!---->        
      
    <p id="ember65" class="ember-view reader-content-blocks__paragraph">
        <ol><li>It should be an Equi-join.</li><li>Hint it as a Merge join for one of the sides.</li></ol>

<!---->    </p>

  

<!---->
<!---->
<!---->        
      
    <p id="ember66" class="ember-view reader-content-blocks__paragraph">
        Example to understand the Shuffle Sort Merge join:

<!---->    </p>

  

<!---->
<!---->
<!---->        
        <pre>spark.conf.set("spark.sql.join.preferSortMergeJoin","true")

orders = spark.read.table("samples.tpch.orders") # Table Size : 272.4MiB 

lineitem = spark.read.table("samples.tpch.lineitem") # Table size : 983MiB 

print("orders",orders.count())
print("lineitem",lineitem.count())

print("orders",orders.rdd.getNumPartitions()) 
print("lineitem",lineitem.rdd.getNumPartitions()) 

orders.join(lineitem,orders.o_orderkey == lineitem.l_orderkey,"inner").explain()</pre>
  

<!---->
<!---->
<!---->        
        <pre>== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- SortMergeJoin [o_orderkey#2385L], [l_orderkey#2419L], Inner
   :- Sort [o_orderkey#2385L ASC NULLS FIRST], false, 0
   :  +- Exchange hashpartitioning(o_orderkey#2385L, 200), ENSURE_REQUIREMENTS, [plan_id=3102]
   :     +- Filter isnotnull(o_orderkey#2385L)
   :        +- FileScan parquet samples.tpch.orders
   +- Sort [l_orderkey#2419L ASC NULLS FIRST], false, 0
      +- Exchange hashpartitioning(l_orderkey#2419L, 200), ENSURE_REQUIREMENTS, [plan_id=3103]
         +- Filter isnotnull(l_orderkey#2419L)
            +- FileScan parquet samples.tpch.lineitem</pre>
  

<!---->
<!---->
<!----><!---->
        
    <div class="reader-image-block reader-image-block--full-width">
      <figure class="reader-image-block__figure">
          
    <div class="ivm-image-view-model   ">
        
    <div class="ivm-view-attr__img-wrapper display-flex">
<!---->          <img src="https://media.licdn.com/dms/image/D5612AQFgLHNBJaTGsA/article-inline_image-shrink_1000_1488/0/1697724924512?e=1707350400&amp;v=beta&amp;t=crNb-gF-cOUZ-MWGuyeWGJDaEPq08G9BnQ6ASw-FX1Q" loading="lazy" alt="" id="ember67" class="ivm-view-attr__img--centered  reader-image-block__img evi-image lazy-image ember-view">
    </div>
  
          </div>
  

          <figcaption class="display-block mt2 full-width text-body-small-open t-sans text-align-center t-black--light">
            <!---->Sort Merge Join stage Implementation<!---->
          </figcaption>
      </figure>
    </div>
  

<!---->
<!---->        
      
    <h3 id="ember68" class="ember-view">
        Cartesian Join

<!---->    </h3>

  

<!---->
<!---->
<!---->        
      
    <p id="ember69" class="ember-view reader-content-blocks__paragraph">
        A Cartesian Join results in each record from one table being joined with every row in the other table. This is a costly join and can lead to out-of-memory (OOM) issues when there is insufficient space to execute the join.

<!---->    </p>

  

<!---->
<!---->
<!---->        
      
    <p id="ember70" class="ember-view reader-content-blocks__paragraph">
        Tips to make this join happen:

<!---->    </p>

  

<!---->
<!---->
<!---->        
      
    <p id="ember71" class="ember-view reader-content-blocks__paragraph">
        <ol><li>It should be a Non-Equi join.</li><li>Joining condition is not necessary when you specify a cross join.</li></ol>

<!---->    </p>

  

<!---->
<!---->
<!---->        
      
    <p id="ember72" class="ember-view reader-content-blocks__paragraph">
        Example to understand the Cartesian Join:

<!---->    </p>

  

<!---->
<!---->
<!---->        
        <pre>orders = spark.read.table("samples.tpch.orders")


lineitem = spark.read.table("samples.tpch.lineitem") 

join_cond = (orders.o_totalprice &lt; lineitem.l_extendedprice)

orders.join(lineitem,join_cond,"inner").explain()</pre>
  

<!---->
<!---->
<!---->        
        <pre>== Physical Plan ==
CartesianProduct Inner, (o_totalprice#507 &lt; l_extendedprice#543)
:- *(1) Filter isnotnull(o_totalprice#507)
:  +- *(1) ColumnarToRow
:     +- FileScan parquet samples.tpch.orders
+- *(2) Filter isnotnull(l_extendedprice#543)
   +- *(2) ColumnarToRow
      +- FileScan parquet samples.tpch.lineitem</pre>
  

<!---->
<!---->
<!---->        
      
    <h3 id="ember73" class="ember-view">
        Broadcast Nested Loop Join

<!---->    </h3>

  

<!---->
<!---->
<!---->        
      
    <p id="ember74" class="ember-view reader-content-blocks__paragraph">
        In a Broadcast Nested Loop Join, we typically have one large table, often a fact table, and a smaller dimension table. Each dimension in the smaller table relates to all the fact records. This dimension data is broadcasted to all the executors handling portions of the fact data.

<!---->    </p>

  

<!---->
<!---->
<!---->        
      
    <p id="ember75" class="ember-view reader-content-blocks__paragraph">
        Tips to make this join happen:

<!---->    </p>

  

<!---->
<!---->
<!---->        
      
    <p id="ember76" class="ember-view reader-content-blocks__paragraph">
        <ol><li>It should be a Non-Equi join.</li><li>Joining condition is not necessary when you specify a cross join.</li><li>The small table can be broadcastable.</li></ol>

<!---->    </p>

  

<!---->
<!---->
<!---->        
        <pre>orders = spark.read.table("samples.tpch.orders").limit(750)

lineitem = spark.read.table("samples.tpch.lineitem") 

join_cond = (orders.o_totalprice &lt; lineitem.l_extendedprice)

orders.join(lineitem,join_cond,"full").explain()</pre>
  

<!---->
<!---->
<!---->        
        <pre>== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- BroadcastNestedLoopJoin BuildLeft, FullOuter, (o_totalprice#1745 &lt; l_extendedprice#1781)
   :- BroadcastExchange IdentityBroadcastMode, [plan_id=1050]
   :  +- GlobalLimit 750, 0
   :     +- Exchange SinglePartition, ENSURE_REQUIREMENTS, [plan_id=1047]
   :        +- LocalLimit 750
   :           +- FileScan parquet samples.tpch.orders
   +- FileScan parquet samples.tpch.lineitem</pre>
  

<!---->
<!---->
<!---->        
      
    <p id="ember77" class="ember-view reader-content-blocks__paragraph">
        Understanding these join types is very vital in understanding Spark workloads and making informed decisions when it comes to choosing the right join strategy if spark approach is not optimal. Being aware of the internals and knowing when to change the join type can significantly impact the performance and efficiency of your Spark applications. ,

<!---->    </p>

  

<!---->
<!---->
<!---->  
      </div>